In [1]:
import time
import os
from tqdm import tqdm

import pandas as pd
import numpy as np

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

# 忽略烦人的红色提示
import warnings
warnings.filterwarnings("ignore")

# 获取计算硬件
# 有 GPU 就用 GPU，没有就用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


In [2]:
from torchvision import transforms

train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                     ])

test_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.485, 0.456, 0.406], 
                                         std=[0.229, 0.224, 0.225])
                                    ])

In [3]:

dataset_dir = '/home/featurize/work/HAM10000'

In [4]:
train_path = os.path.join(dataset_dir, 'train')
test_path = os.path.join(dataset_dir, 'val')
print('训练集路径', train_path)
print('测试集路径', test_path)

from torchvision import datasets

train_dataset = datasets.ImageFolder(train_path, train_transform)

test_dataset = datasets.ImageFolder(test_path, test_transform)

print('训练集图像数量', len(train_dataset))
print('类别个数', len(train_dataset.classes))
print('各类别名称', train_dataset.classes)
print('测试集图像数量', len(test_dataset))
print('类别个数', len(test_dataset.classes))
print('各类别名称', test_dataset.classes)

训练集路径 /home/featurize/work/HAM10000/train
测试集路径 /home/featurize/work/HAM10000/val
训练集图像数量 21255
类别个数 7
各类别名称 ['AKIEC', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'VASC']
测试集图像数量 5310
类别个数 7
各类别名称 ['AKIEC', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'VASC']


In [5]:

class_names = train_dataset.classes
n_class = len(class_names)

train_dataset.class_to_idx

idx_to_labels = {y:x for x,y in train_dataset.class_to_idx.items()}

In [6]:
idx_to_labels

{0: 'AKIEC', 1: 'BCC', 2: 'BKL', 3: 'DF', 4: 'MEL', 5: 'NV', 6: 'VASC'}

In [7]:

np.save('idx_to_labels.npy', idx_to_labels)
np.save('labels_to_idx.npy', train_dataset.class_to_idx)

In [8]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32


train_loader = DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=4
                         )


test_loader = DataLoader(test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False,
                         num_workers=4
                        )

In [15]:


class Inception(nn.Module):
    def __init__(self, in_channels):
        super(Inception, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)

        self.branch3x3_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3_2 = nn.Conv2d(16, 16, kernel_size=3, padding=1)

        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.branch5x5_3 = nn.Conv2d(32, 32, kernel_size=3, padding=1)

        self.branch_pool = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.branch_pool_conv = nn.Conv2d(in_channels, 16, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch3x3 = self.branch3x3_1(x)
        branch3x3 = self.branch3x3_2(branch3x3)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)
        branch5x5 = self.branch5x5_3(branch5x5)

        branch_pool = self.branch_pool(x)
        branch_pool = self.branch_pool_conv(branch_pool)

        outputs = [branch1x1, branch3x3, branch5x5, branch_pool]
        return torch.cat(outputs, 1)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, num_layers):
        super(ResidualBlock, self).__init__()
        self.blocks = nn.ModuleList()
        for _ in range(num_layers):
            self.blocks.append(nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
                nn.SELU(inplace=True)
            ))
            in_channels = out_channels # for subsequent layers

        self.shortcut = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
            nn.SELU(inplace=True)
        )

    def forward(self, x):
        residual = x
        for block in self.blocks:
            x = block(x)
        residual = self.shortcut(residual)
        x += residual
        return F.selu(x, inplace=True)

class Inception_ResNet(nn.Module):
    def __init__(self):
        super(Inception_ResNet, self).__init__()
        self.inception = Inception(in_channels=3)
        self.resnet_1x3 = ResidualBlock(64, 256, kernel_size=3, stride=1, padding=1, num_layers=3)
        self.resnet_2x4 = ResidualBlock(256, 128, kernel_size=3, stride=1, padding=1, num_layers=2)
        self.resnet_3x6 = ResidualBlock(128, 1024, kernel_size=3, stride=1, padding=1, num_layers=6)
        self.resnet_4x3 = ResidualBlock(1024, 512, kernel_size=3, stride=1, padding=1, num_layers=3)
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.fc = nn.Linear(512 * 7 * 7, 7) 

    def forward(self, x):
        x = self.inception(x)
        x = self.resnet_1x3(x)
        x = self.resnet_2x4(x)
        x = self.resnet_3x6(x)
        x = self.resnet_4x3(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Instantiate the model
model = Inception_ResNet()

# Model summary
print(model)



model = Inception_ResNet()


optimizer = optim.Adam(model.parameters())



In [16]:
model = model.to(device)


criterion = nn.CrossEntropyLoss() 


EPOCHS = 30


lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [17]:
!pip install scikit-learn
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [18]:
def train_one_batch(images, labels):
 

    images = images.to(device)
    labels = labels.to(device)
    
    outputs = model(images) 
    loss = criterion(outputs, labels) 
    
  
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    

    _, preds = torch.max(outputs, 1) 
    preds = preds.cpu().numpy()
    loss = loss.detach().cpu().numpy()
    outputs = outputs.detach().cpu().numpy()
    labels = labels.detach().cpu().numpy()
    
    log_train = {}
    log_train['epoch'] = epoch
    log_train['batch'] = batch_idx
  
    log_train['train_loss'] = loss
    log_train['train_accuracy'] = accuracy_score(labels, preds)
   
    
    return log_train

In [19]:
def evaluate_testset():
  
    loss_list = []
    labels_list = []
    preds_list = []
    
    with torch.no_grad():
        for images, labels in test_loader: 
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images) 

           
            _, preds = torch.max(outputs, 1)
            preds = preds.cpu().numpy()
            loss = criterion(outputs, labels) 
            loss = loss.detach().cpu().numpy()
            outputs = outputs.detach().cpu().numpy()
            labels = labels.detach().cpu().numpy()

            loss_list.append(loss)
            labels_list.extend(labels)
            preds_list.extend(preds)
        
    log_test = {}
    log_test['epoch'] = epoch
    

    log_test['test_loss'] = np.mean(loss_list)
    log_test['test_accuracy'] = accuracy_score(labels_list, preds_list)
    log_test['test_precision'] = precision_score(labels_list, preds_list, average='macro')
    log_test['test_recall'] = recall_score(labels_list, preds_list, average='macro')
    log_test['test_f1-score'] = f1_score(labels_list, preds_list, average='macro')
    
    return log_test

In [20]:
epoch = 0
batch_idx = 0
best_test_accuracy = 0

In [21]:

df_train_log = pd.DataFrame()
log_train = {}
log_train['epoch'] = 0
log_train['batch'] = 0
images, labels = next(iter(train_loader))
log_train.update(train_one_batch(images, labels))
df_train_log = df_train_log._append(log_train, ignore_index=True)

In [22]:
df_train_log

,epoch,batch,train_loss,train_accuracy
0,0,0,1.9233159,0.125


In [23]:
# 训练日志-测试集
df_test_log = pd.DataFrame()
log_test = {}
log_test['epoch'] = 0
log_test.update(evaluate_testset())
df_test_log = df_test_log._append(log_test, ignore_index=True)

In [24]:
df_test_log

,epoch,test_loss,test_accuracy,test_precision,test_recall,test_f1-score
0,0.0,2.547374,0.230697,0.13465,0.151779,0.138029


In [25]:
try:
    from typing import Literal
except ImportError:
    from typing_extensions import Literal
import wandb


wandb.init(project='Skin_cancer', name=time.strftime('%m%d%H%M%S'))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mengxiangyu014. Use `wandb login --relogin` to force relogin


In [26]:
for epoch in range(1, EPOCHS+1):
    
    print(f'Epoch {epoch}/{EPOCHS}')
    

    model.train()
    for images, labels in tqdm(train_loader): 
        batch_idx += 1
        log_train = train_one_batch(images, labels)
        df_train_log = df_train_log._append(log_train, ignore_index=True)
        wandb.log(log_train)
        
    lr_scheduler.step()


    model.eval()
    log_test = evaluate_testset()
    df_test_log = df_test_log._append(log_test, ignore_index=True)
    wandb.log(log_test)
    
   
    if log_test['test_accuracy'] > best_test_accuracy: 
      
        old_best_checkpoint_path = 'checkpoint/best-{:.3f}.pth'.format(best_test_accuracy)
        if os.path.exists(old_best_checkpoint_path):
            os.remove(old_best_checkpoint_path)
      
        best_test_accuracy = log_test['test_accuracy']
        new_best_checkpoint_path = 'checkpoint/best-{:.3f}.pth'.format(log_test['test_accuracy'])
        torch.save(model, new_best_checkpoint_path)
        print('保存新的最佳模型', 'checkpoint/best-{:.3f}.pth'.format(best_test_accuracy))
     

df_train_log.to_csv('训练日志-训练集.csv', index=False)
df_test_log.to_csv('训练日志-测试集.csv', index=False)

Epoch 1/30


100%|██████████| 665/665 [03:43<00:00,  2.97it/s]


保存新的最佳模型 checkpoint/best-0.551.pth
Epoch 2/30


100%|██████████| 665/665 [03:45<00:00,  2.95it/s]


保存新的最佳模型 checkpoint/best-0.584.pth
Epoch 3/30


...

保存新的最佳模型 checkpoint/best-0.863.pth
Epoch 29/30


100%|██████████| 665/665 [03:48<00:00,  2.91it/s]


保存新的最佳模型 checkpoint/best-0.870.pth
Epoch 30/30


100%|██████████| 665/665 [04:09<00:00,  2.67it/s]


保存新的最佳模型 checkpoint/best-0.872.pth


In [28]:

model = torch.load('checkpoint/best-{:.3f}.pth'.format(best_test_accuracy))

In [29]:
model.eval()
print(evaluate_testset())

{'epoch': 30, 'test_loss': 0.34036866, 'test_accuracy': 0.871939736346516, 'test_precision': 0.8792434679963509, 'test_recall': 0.8798923446960123, 'test_f1-score': 0.8791445234803907}
